# Iris Training and Prediction with Sagemaker Scikit-learn
 이 자습서에서는 미리 빌드된 Scikit-learn [Scikit-learn](https://scikit-learn.org/stable/) 컨테이너를 사용하여 Sagemaker에서 를 사용하는 방법을 보여 줍니다.Scikit-learn은 인기있는 파이썬 기계 학습 프레임 워크입니다.여기에는 분류, 회귀, 클러스터링, 차원 감소 및 데이터/기능 전처리를위한 다양한 알고리즘이 포함됩니다.  

[sagemaker-python-sdk](https://github.com/aws/sagemaker-python-sdk) 모듈을 사용하면 기존 scikit-learn 코드를 쉽게 가져올 수 있습니다. 이 코드는 IRIS 데이터 세트에 대한 모델을 훈련하고 예측 집합을 생성하여 보여줍니다. Scikit-learn container에 대한 더 많은 정보는 이곳을 확인하십시오. [sagemaker-scikit-learn-containers](https://github.com/aws/sagemaker-scikit-learn-container) repository and the [sagemaker-python-sdk](https://github.com/aws/sagemaker-python-sdk)

Scikit-learn 에 대한 더 자세한 정보는 다음을 확인하십시오: <http://scikit-learn.org/stable/>.

### Table of contents
* [Upload the data for training](#upload_data)
* [Create a Scikit-learn script to train with](#create_sklearn_script)
* [Create the SageMaker Scikit Estimator](#create_sklearn_estimator)
* [Train the SKLearn Estimator on the Iris data](#train_sklearn)
* [Using the trained model to make inference requests](#inferece)
 * [Deploy the model](#deploy)
 * [Choose some data and use it for a prediction](#prediction_request)
 * [Endpoint cleanup](#endpoint_cleanup)
* [Batch Transform](#batch_transform)
 * [Prepare Input Data](#prepare_input_data)
 * [Run Transform Job](#run_transform_job)
 * [Check Output Data](#check_output_data)

**Note: this example requires SageMaker Python SDK v2.**

먼저 Sagemaker session 과 role 을 만들고 노트북 예제에 사용할 S3 prefix 를 만듭니다.

In [23]:
# S3 prefix
prefix = "byos/scikit-iris"
bucket = 'yudong-data'

import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

# Get a SageMaker-compatible role used by this Notebook Instance.
role = get_execution_role()

Sagemaker version 을 체크합니다. 본 예제는 SageMaker Python SDK v2 이상을 요구합니다.

In [5]:
sagemaker.__version__

'2.45.0'

만약에 v2 이하의 버젼인 경우 아래 코드의 주석을 제거하고 실행합니다. v2 에 대한 자세한 정보는 [다음](https://sagemaker.readthedocs.io/en/stable/v2.html) 에서 확인할 수 있습니다.

In [6]:
# %pip install -U sagemaker>=2.15

## Upload the data for training <a class="anchor" id="upload_data"></a>

방대한 양의 데이터가 포함된 대규모 모델을 훈련할 때는 일반적으로 Amazon Athena, AWS Gluse 또는 Amazon EMR과 같은 빅 데이터 도구를 사용하여 S3에 데이터를 생성합니다. 이 예제에서는 Scikit-learn에 포함된 아이리스 데이터 [Iris dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set)를 사용합니다. 데이터를 로드하고 로컬로 작성한 다음 데이터를 s3에 씁니다.

In [8]:
import numpy as np
import os
from sklearn import datasets

# Load Iris dataset, then join labels and features
iris = datasets.load_iris()
joined_iris = np.insert(iris.data, 0, iris.target, axis=1)

# Create directory and write csv
os.makedirs("./data", exist_ok=True)
np.savetxt("./data/iris.csv", joined_iris, delimiter=",", fmt="%1.1f, %1.3f, %1.3f, %1.3f, %1.3f")

iris 데이터 셋은 다음 컬럼 정보를 가지고 있습니다. Sepal Length, Sepal Width, Petal Length and Petal Width. 자세한 정보는 [scikit-learn iris data](https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html) 페이지를 참조하시기 바랍니다.

In [16]:
joined_iris[:10]

array([[0. , 5.1, 3.5, 1.4, 0.2],
       [0. , 4.9, 3. , 1.4, 0.2],
       [0. , 4.7, 3.2, 1.3, 0.2],
       [0. , 4.6, 3.1, 1.5, 0.2],
       [0. , 5. , 3.6, 1.4, 0.2],
       [0. , 5.4, 3.9, 1.7, 0.4],
       [0. , 4.6, 3.4, 1.4, 0.3],
       [0. , 5. , 3.4, 1.5, 0.2],
       [0. , 4.4, 2.9, 1.4, 0.2],
       [0. , 4.9, 3.1, 1.5, 0.1]])

데이터를 로컬에 저장하면 SageMaker Python SDK에서 제공하는 도구를 사용하여 데이터를 기본 버킷에 업로드할 수 있습니다. 

In [24]:
# WORK_DIRECTORY = "data"

train_input = sagemaker_session.upload_data(
    './data/iris.csv', bucket=bucket, key_prefix=prefix
)

## Create a Scikit-learn script to train with <a class="anchor" id="create_sklearn_script"></a>
SageMaker는 'Sklearn' Estimator 를 사용하여 Scikit-learn 훈련 스크립트를 실행할 수 있습니다. SageMaker 에서 실행될 때 여러 유용한 환경 변수를 사용하여 교육 환경의 속성에 액세스할 수 있습니다. 예를 들면,

* `SM_MODEL_DIR`: 모델 객체를 쓸 디렉토리의 경로를 나타내는 문자열입니다. 이 폴더에 저장된 모든 객체는 교육 작업이 완료된 후 모델 호스팅을 위해 S3에 업로드됩니다.
* `SM_OUTPUT_DIR`: output 아티팩트를 쓸 파일 시스템 경로를 나타내는 문자열입니다. output 아티팩트에는 checkpoints, 그래프 및 저장할 기타 파일 (모델 아티팩트 제외) 이 포함될 수 있습니다. 이러한 아티팩트는 모델 아티팩트와 동일한 S3 접두사로 압축되어 S3에 업로드됩니다.

'sklearn' Estimator 의 'fit ()' 메소드를 호출 할 때 두 개의 입력 채널 인 'train '과 'test'가 사용되었다고 가정하면 다음과 같은 환경 변수가 설정됩니다. `SM_CHANNEL_[channel_name]`:

* `SM_CHANNEL_TRAIN`: 'train' 채널의 데이터를 포함하는 디렉토리 경로를 나타내는 문자열
* `SM_CHANNEL_TEST`: 'test' 채널

일반적인 훈련 스크립트는 입력 채널에서 데이터를 로드하고, 하이퍼파라미터를 사용하여 훈련을 구성하고, 모델을 학습하고, 모델을 model_dir 에 저장하여 나중에 호스팅할 수 있습니다. 하이퍼 매개 변수는 인수로 스크립트에 전달되고`argparse.argumentParser` 인스턴스로 검색 할 수 있습니다. 아래 예제를 참고합니다.

```python
from __future__ import print_function

import argparse
import joblib
import os
import pandas as pd

from sklearn import tree


if __name__ == '__main__':
    """
    Train a Random Forest Regressor
    """
    print("Training mode")

    try:
        X_train, y_train = load_dataset(args.train)
        X_test, y_test = load_dataset(args.test)

        hyperparameters = {
            "max_depth": args.max_depth,
            "verbose": 1,  # show all logs
            "n_jobs": args.n_jobs,
            "n_estimators": args.n_estimators,
        }
        print("Training the classifier")
        model = RandomForestRegressor()
        model.set_params(**hyperparameters)
        model.fit(X_train, y_train)
        print("Score: {}".format(model.score(X_test, y_test)))
        # joblib.dump(model, open(os.path.join(args.model_dir, "iris_model.pkl"), "wb"))
        joblib.dump(model, os.path.join(args.model_dir, "model.joblib"))

    except Exception as e:
        # Write out an error file. This will be returned as the failureReason in the
        # DescribeTrainingJob result.
        trc = traceback.format_exc()
        with open(os.path.join(output_path, "failure"), "w") as s:
            s.write("Exception during training: " + str(e) + "\\n" + trc)

        # Printing this causes the exception to be in the training job logs, as well.
        print("Exception during training: " + str(e) + "\\n" + trc, file=sys.stderr)

        # A non-zero exit code causes the training job to be marked as Failed.
        sys.exit(255)


def model_fn(model_dir):
    """Deserialized and return fitted model
    
    Note that this should have the same name as the serialized model in the main method
    """
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf
```

In [25]:
%%writefile scikit_learn_iris.py

from __future__ import print_function

import argparse
import joblib
import os
import pandas as pd

from sklearn import tree


if __name__ == '__main__':
    parser = argparse.ArgumentParser()

    # Hyperparameters are described here. In this simple example we are just including one hyperparameter.
    # TODO: parser 를 사용하여 max_leaf_nodes 를 불러옵니다. Option: default 는 -1 로 정해줄수 있습니다.
    parser.add_argument()

    # Sagemaker specific arguments. Defaults are set in the environment variables.
    parser.add_argument('--output-data-dir', type=str, default=os.environ['SM_OUTPUT_DATA_DIR'])
    parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])
    parser.add_argument('--train', type=str, default=os.environ['SM_CHANNEL_TRAIN'])

    args = parser.parse_args()

    # Take the set of files and read them all into a single pandas dataframe
    input_files = [ os.path.join(args.train, file) for file in os.listdir(args.train) ]
    if len(input_files) == 0:
        raise ValueError(('There are no files in {}.\n' +
                          'This usually indicates that the channel ({}) was incorrectly specified,\n' +
                          'the data specification in S3 was incorrectly specified or the role specified\n' +
                          'does not have permission to access the data.').format(args.train, "train"))
    raw_data = [ pd.read_csv(file, header=None, engine="python") for file in input_files ]
    train_data = pd.concat(raw_data)

    # labels are in the first column
    # TODO: train 을 위한 feature 변수와 label 변수를 분리합니다.
    train_X = 
    train_y =
    
    # Here we support a single hyperparameter, 'max_leaf_nodes'. Note that you can add as many
    # as your training my require in the ArgumentParser above.
    # TODO: argument 변수에서 max_leaf_nodes 를 불러와서 변수를 만듭니다.
    max_leaf_nodes = 

    # Now use scikit-learn's decision tree classifier to train the model.
    clf = tree.DecisionTreeClassifier(max_leaf_nodes=max_leaf_nodes)
    clf = clf.fit(train_X, train_y)

    # Print the coefficients of the trained classifier, and save the coefficients
    joblib.dump(clf, os.path.join(args.model_dir, "model.joblib"))


def model_fn(model_dir):
    """Deserialized and return fitted model
    
    Note that this should have the same name as the serialized model in the main method
    """
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf

Writing scikit_learn_iris.py


Scikit-Learn 컨테이너는 훈련 스크립트를 가져 오기 때문에 컨테이너가 실수로 훈련 코드를 실행하지 않도록 항상 \__name__ == \__main__ 에 훈련 코드를 넣어야합니다.

환경 변수에 대한 더 자세한 정보는 다음을 살펴보시기 바랍니다. https://github.com/aws/sagemaker-containers.

## Create SageMaker Scikit Estimator <a class="anchor" id="create_sklearn_estimator"></a>

SgeMaker에서 Scikit-Learn 훈련 스크립트를 실행하기 위해, 우리는 몇 가지 생성자 인수를 받아들이는 `sagemaker.sklearn.estimator.sklearn` Estimator 를 구성합니다:

* __entry_point__: SageMaker는 학습 및 예측을 위해 실행되는 파이썬 스크립트 경로입니다.
* __role__: Role ARN
* __instance_type__ *(optional)*: 학습을 위한 SageMaker 인스턴스의 유형입니다. __Note__: Because Scikit-learn does not natively support GPU training, Sagemaker Scikit-learn does not currently support training on GPU instance types.
* __sagemaker_session__ *(optional)*: The session used to train on Sagemaker.
* __hyperparameters__ *(optional)*: A dictionary passed to the train function as hyperparameters.

To see the code for the SKLearn Estimator, see here: https://github.com/aws/sagemaker-python-sdk/tree/master/src/sagemaker/sklearn

In [44]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"
script_path = "scikit_learn_iris.py"

sklearn = SKLearn(
    entry_point=script_path,
    framework_version=FRAMEWORK_VERSION,
    instance_type="ml.c4.xlarge",
    role=role,
    sagemaker_session=sagemaker_session,
    hyperparameters={"max_leaf_nodes": 30},
)

## Train SKLearn Estimator on Iris data <a class="anchor" id="train_sklearn"></a>
교육은 매우 간단합니다. Estimator 에 fit 이라는 함수를 호출합니다! 그러면 SageMaker 훈련 작업이 시작되어 데이터를 다운로드하고, 제공된 스크립트 파일에서 scikit-learn 코드를 호출하고, 스크립트가 생성하는 모든 모델 아티팩트를 저장합니다.

In [ ]:
sklearn.fit({"train": train_input})

## Using the trained model to make inference requests <a class="anchor" id="inference"></a>

### Deploy the model <a class="anchor" id="deploy"></a>

모델을 SageMaker 호스팅에 배포하려면 적합 모델에 대한 'deploy' 함수 호출이 필요합니다. 이 호출은 인스턴스 수와 인스턴스 유형을 사용합니다.

In [28]:
predictor = sklearn.deploy(initial_instance_count=1, instance_type="ml.m5.xlarge")

-------------!

### Choose some data and use it for a prediction <a class="anchor" id="prediction_request"></a>

몇 가지 예측을 수행하기 위해 교육에 사용한 일부 데이터를 추출하고 이에 대한 예측을 수행합니다. 이것은 물론 나쁜 통계적 관행이지만 메커니즘이 어떻게 작동하는지 확인하는 좋은 방법입니다.

In [29]:
import itertools
import pandas as pd

shape = pd.read_csv("data/iris.csv", header=None)

a = [50 * i for i in range(3)]
b = [40 + i for i in range(10)]
indices = [i + j for i, j in itertools.product(a, b)]

test_data = shape.iloc[indices[:-1]]
test_X = test_data.iloc[:, 1:]
test_y = test_data.iloc[:, 0]

예측은 배포에서 돌아온 예측 변수와 예측을 수행하려는 데이터로 예측을 호출하는 것만큼 쉽습니다. Endpoint의 출력은 분류 예측의 숫자 표현을 반환합니다. 원래 데이터 집합에서 이들은 꽃 이름이지만 이 예에서는 레이블은 숫자입니다.우리는 우리가 파싱 한 원래 레이블과 비교할 수 있습니다.

In [31]:
prediction = predictor.predict(test_X.values)

In [35]:
from sklearn.metrics import classification_report, make_scorer, accuracy_score, precision_score
from sklearn.metrics import confusion_matrix

In [37]:
# TODO: confusion matrix/classification report 를 출력해봅니다.

### Endpoint cleanup <a class="anchor" id="endpoint_cleanup"></a>

Endpoint 작업이 완료되면 정리할 수 있습니다.

In [38]:
predictor.delete_endpoint()

## Batch Transform <a class="anchor" id="batch_transform"></a>
또한 SageMaker Batch transform 을 사용하여 S3 데이터에 대한 비동기 배치 추론을 위해 훈련된 모델을 사용할 수 있습니다.

In [45]:
# Define a SKLearn Transformer from the trained SKLearn Estimator
transformer = sklearn.transformer(instance_count=1, instance_type="ml.m5.xlarge")

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


### Prepare Input Data <a class="anchor" id="prepare_input_data"></a>
학습 데이터에서 100 행의 무작위 샘플을 10 개 추출한 다음 레이블 (Y) 에서 피처 (X) 를 분할합니다. 그런 다음 입력 데이터를 S3의 지정된 위치에 업로드합니다.

In [46]:
%%bash
# Randomly sample the iris dataset 10 times, then split X and Y
mkdir -p batch_data/XY batch_data/X batch_data/Y
for i in {0..9}; do
    cat data/iris.csv | shuf -n 100 > batch_data/XY/iris_sample_${i}.csv
    cat batch_data/XY/iris_sample_${i}.csv | cut -d',' -f2- > batch_data/X/iris_sample_X_${i}.csv
    cat batch_data/XY/iris_sample_${i}.csv | cut -d',' -f1 > batch_data/Y/iris_sample_Y_${i}.csv
done

In [47]:
# Upload input data from local filesystem to S3
batch_input_s3 = sagemaker_session.upload_data("batch_data/X", key_prefix=prefix + "/batch_input")

In [50]:
batch_input_s3

's3://sagemaker-ap-northeast-2-806174985048/byos/scikit-iris/batch_input'

### Run Transform Job <a class="anchor" id="run_transform_job"></a>
Transformer 를 사용하여 S3 입력 데이터에 대해 변환 작업을 실행합니다.

In [ ]:
# Start a transform job and wait for it to finish
transformer.transform(batch_input_s3, content_type="text/csv")
print("Waiting for transform job: " + transformer.model_name)
transformer.wait()

### Check Output Data  <a class="anchor" id="check_output_data"></a>
변환 작업이 완료되면 S3에서 출력 데이터를 다운로드합니다. 입력 데이터의 각 파일 “F”에 대해 각 입력 행에서 예측 된 레이블을 포함하는 해당 파일 “f.out”이 있습니다. 우리는 이전에 저장된 실제 레이블과 예측 된 레이블을 비교할 수 있습니다.

In [56]:
# Download the output data from S3 to local filesystem
batch_output = transformer.output_path
!mkdir -p batch_data/output
!aws s3 cp --recursive $batch_output/ batch_data/output/
# Head to see what the batch output looks like
!head batch_data/output/*

head: cannot open ‘batch_data/output/*’ for reading: No such file or directory


In [ ]:
%%bash
# For each sample file, compare the predicted labels from batch output to the true labels
for i in {1..9}; do
    diff -s batch_data/Y/iris_sample_Y_${i}.csv \
        <(cat batch_data/output/iris_sample_X_${i}.csv.out | sed 's/[["]//g' | sed 's/, \|]/\n/g') \
        | sed "s/\/dev\/fd\/63/batch_data\/output\/iris_sample_X_${i}.csv.out/"
done

In [ ]:
## SKLearn 을 사용한 Ridge 모델 예제
'''
DERIVED FROM:https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/sklearn/README.rst
Preparing the Scikit-learn training script
Your Scikit-learn training script must be a Python 2.7 or 3.5 compatible source file.
The training script is very similar to a training script you might run outside of SageMaker, 
but you can access useful properties about the training environment through various environment variables, 
such as
- SM_MODEL_DIR: 
        A string representing the path to the directory to write model artifacts to. 
        These artifacts are uploaded to S3 for model hosting.
- SM_OUTPUT_DATA_DIR: 
        A string representing the filesystem path to write output artifacts to. 
        Output artifacts may include checkpoints, graphs, and other files to save, 
        not including model artifacts. These artifacts are compressed and uploaded 
        to S3 to the same S3 prefix as the model artifacts.
        Supposing two input channels, 'train' and 'test', 
        were used in the call to the Scikit-learn estimator's fit() method, 
        the following will be set, following the format "SM_CHANNEL_[channel_name]":
- SM_CHANNEL_TRAIN: 
        A string representing the path to the directory containing data in the 'train' channel
- SM_CHANNEL_TEST: 
        Same as above, but for the 'test' channel.
        A typical training script loads data from the input channels, 
        configures training with hyperparameters, trains a model, 
        and saves a model to model_dir so that it can be hosted later. 
        Hyperparameters are passed to your script as arguments and can 
        be retrieved with an argparse.ArgumentParser instance. 
        For example, a training script might start with the following:
Because the SageMaker imports your training script, 
you should put your training code in a main guard (if __name__=='__main__':) 
if you are using the same script to host your model, 
so that SageMaker does not inadvertently run your training code at the wrong point in execution.
For more on training environment variables, please visit https://github.com/aws/sagemaker-containers.
'''


import argparse
import os

import pandas as pd

import sklearn

from sklearn import linear_model

import numpy as np

import six
from six import StringIO, BytesIO

# from sagemaker.content_types import CONTENT_TYPE_JSON, CONTENT_TYPE_CSV, CONTENT_TYPE_NPY
# Interesting fact: 
#   on SageMaker model training instance, py-sagemaker is not installed
# import sagemaker 

# matplotlib is not available 
# from matplotlib import pyplot as plt

from sklearn.externals import joblib
import json 

from sagemaker_containers.beta.framework import (
    content_types, encoders, env, modules, transformer, worker)


fn = 'sample_data.json'

MA_list = [50, 100, 200, 300, 400, 800, 1600]
intput_col = ["MA-{}".format(ma_lag) for ma_lag in MA_list[:-1]]
benchmark_col = 'MA-1600'


'''
The RealTimePredictor used by Scikit-learn in the SageMaker 
Python SDK serializes NumPy arrays to the NPY format by default, 
with Content-Type application/x-npy. The SageMaker Scikit-learn model server 
can deserialize NPY-formatted data (along with JSON and CSV data).
'''
def input_fn(request_body, request_content_type):
    """An input_fn that loads a pickled numpy array"""
    # print("request_body=",str(request_body))
    # print("np.load(StringIO(request_body))=",np.load(StringIO(request_body)))

    if request_content_type == "application/python-pickle":
        array = np.load(BytesIO((request_body)))
        # print("array=",array)
        return array
    elif request_content_type == 'application/json':
        jsondata = json.load(StringIO(request_body))
        normalized_data, benchmark_data = process_input_data(jsondata)
        # print("normalized_data=",normalized_data)
        return normalized_data, benchmark_data
    else:
        # Handle other content-types here or raise an Exception
        # if the content type is not supported.
        raise ValueError("{} not supported by script!".format(request_content_type))

def output_fn(prediction, accept):
    """Format prediction output
    The default accept/content-type between containers for serial inference is JSON.
    We also want to set the ContentType or mimetype as the same value as accept so the next
    container can read the response payload correctly.
    """
    if accept == "application/json":
        return worker.Response(json.dumps(prediction), accept, mimetype=accept)
    elif accept == 'text/csv':
        return worker.Response(encoders.encode(prediction, accept), accept, mimetype=accept)
    else:
        raise ValueError("{} accept type is not supported by this script.".format(accept))

def predict_fn(input_data, model):
    """Preprocess input data
    We implement this because the default predict_fn uses .predict(), but our model is a preprocessor
    so we want to use .transform().
    The output is returned in the following order:
        rest of features either one hot encoded or standardized
    """
    normalized_data, benchmark_data = input_data
    
    prediction = model.predict(normalized_data)
    
    output = np.array(prediction) * np.array(benchmark_data)
    
    return {'prediction-base-time': str(normalized_data.index[-1]), 
            'predicted-value': output[-1]}

def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf

def process_input_data(cmcjsondata, for_training = False):
    raw_data = pd.DataFrame(cmcjsondata)
    dat = pd.DataFrame(list(raw_data['price_usd']), columns=['timestamp', 'usd'])
    dat['dt_utc'] = pd.to_datetime(dat['timestamp']*1e6)
    dat = dat.set_index('dt_utc')
    resampled_data = dat['usd'].resample('30S').mean().interpolate('linear')

    feature = {}

    if for_training:
        Y = resampled_data.rolling(2880).median().shift(-2879) # next 24 hour
        feature['Y'] = Y

    for ma_lag in MA_list:
        feature["MA-{}".format(ma_lag)] = resampled_data.rolling(ma_lag).mean()

    data = pd.DataFrame(feature).dropna()
    benchmark_data = data[benchmark_col].copy()
    normalized_data = data.div(data[benchmark_col], axis=0)
    
    _col = intput_col + (['Y'] if for_training else [])

    return normalized_data[_col], benchmark_data
    
def run_training(args):

    with open(os.path.join(args.train, fn)) as fp:
        jsondata = json.load(fp)
    normalized_data, _ = process_input_data(jsondata, for_training = True)

    model = linear_model.Ridge()
    model.fit(normalized_data[intput_col], normalized_data['Y'])

    joblib.dump(model, os.path.join(args.model_dir, "model.joblib"))


if __name__ =='__main__':

    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script.
    parser.add_argument('--epochs', type=int, default=50)
    parser.add_argument('--batch-size', type=int, default=64)
    parser.add_argument('--learning-rate', type=float, default=0.05)

    # Data, model, and output directories
    parser.add_argument('--output-data-dir', type=str, default=os.environ.get('SM_OUTPUT_DATA_DIR'))
    parser.add_argument('--model-dir', type=str, default=os.environ.get('SM_MODEL_DIR'))
    parser.add_argument('--train', type=str, default=os.environ.get('SM_CHANNEL_TRAIN'))
    parser.add_argument('--test', type=str, default=os.environ.get('SM_CHANNEL_TEST'))

    args, _ = parser.parse_known_args()

    run_training(args)

    # ... load from args.train and args.test, train a model, write model to args.model_dir.